In [29]:
import pandas as pd
import re
import rdflib


In [30]:
##Enter the collection name formulated to also be a name for the linked data spreadsheet
collection = 'Spelman_Linked_Data'
##Read in your metadata csv file
meta = pd.read_csv("SpelmanSubset.csv",skiprows=1, header=None)
##List your column data types
headers=["ID", "Title", "Date", "Description", "LCSH", "Geoname", "Taken at"]
#Enter the number for the column containing item ID data (Start counting at zero)
idcol=0
#Enter the number for the column containing item date data (Start counting at zero)
datecol=2
#Enter the number for the column containing item titles (Start counting at zero)
titles=1
#Enter the number for the column containing item description (Start counting at zero)
desc=1
#Enter the number for the column containing location data (name) (Start counting at zero)
place=6

triples=[]
desc_list=[]
t_list=[]
for item,line in meta.iterrows():
    datedata = line[datecol]
    #customize regex as needed to locate the item ID cell in each row
    itemid = line[0]
    row=meta.loc[item, :].values.flatten().tolist()
    row = ''.join(str(row))
    clearid="\'"+str(itemid)+"\', "
    row = re.sub(clearid, "", row)
    #Adjust here depending on how your subject tags are divided
    row = re.sub(r"\<br\/\>", "\', \'", row)
    #Any additional cleaning and standardization can be done here
    row = re.sub("\/\S*\s"," ", row)
    row = re.sub("``","\"", row)
    row = re.sub("\'\'","\"", row)
    row = re.sub("\/\'\'", "", row)
    desc_list=row.split("\', \'")

    for term in desc_list:
        term = re.sub('"',"",term)
        idnum = row[idcol]
    if re.finditer("\s", idnum) is True:
        re.sub("\s", "_", idnum)
    triples.append([idnum,"http://www.w3.org/2008/05/skos#Collection", collection])
    idtitle = row[titles]
    if re.finditer("\s", idtitle) is True:
        re.sub("\s", "_", idtitle)
    triples.append([idtitle, "http://www.w3.org/2008/05/skos#Collection", collection])
    triples.append([idnum, "http://www.w3.org/2008/05/skos#exactMatch", idtitle])
    triples.append([idnum, "http://www.w3.org/2008/05/skos#exactMatch", row[place]])
    triples.append([desc, "http://purl.org/dc/elements/1.1/description", idnum])
    listlength=len(desc_list)
    listlength-=1
    while listlength>0:
        t_list=[itemid, "contains", desc_list[listlength].strip()]
        listlength-=1
        triples.append(t_list)
        t_list = [itemid, "http://purl.org/dc/terms/date", datedata]
        triples.append(t_list)



In [31]:
#Import boxes.csv(generated)
persons = pd.read_csv("autodrawboxes.csv",skiprows=1, header=None)
##Set the first unique ID number to use for those depicted within the image collection
counter=10001
for index,row in persons.iterrows():
    p_list = []
    itemid = re.match(r"3A(.*?),", str(row))
    p_list = [itemid, "http://xmlns.com/foaf/0.1/depicts", counter]
    triples.append(p_list)
    counter+=1

In [28]:
#Import list of people named in item descriptions in csv format with a column for the itemid and a column for names (if had)
#names = pd.read_csv("names.csv",skiprows=1, header=None)
#for index,row in names.iterrows():
#    p_list = []
#    itemid = re.match(r"3A(.*?),", str(row))
#    p_list = [itemid, "http://xmlns.com/foaf/0.1/depicts", row[1]]
#    triples.append(p_list)

In [26]:
#where exact person's name is known, import list of people named in item descriptions in csv format with a column for the itemid and a column for personid match (if had)
#names = pd.read_csv("names.csv",skiprows=1, header=None)
#for index,row in names.iterrows():
#    p_list = []
#    itemid = re.match(r"3A(.*?),", str(row))
#    p_list =([row[0], "http://www.w3.org/2008/05/skos#exactMatch", row[1])
#    triples.append(p_list)

In [ ]:
df = pd.DataFrame(triples, columns = ['Subject', 'Predicate', 'Object'])
df.to_csv(str(collection)+'tripleslist.csv', index=False)